In [1]:
# Initializations
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from matplotlib import pyplot as plt
import collections
import nest_asyncio
%reload_ext tensorboard
nest_asyncio.apply()

In [3]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
emnist_train.element_type_structure

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)),
             ('pixels',
              TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])

In [4]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER= 10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [5]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))
sample_batch

OrderedDict([('x',
              array([[1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.],
                     ...,
                     [1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)),
             ('y',
              array([[3],
                     [7],
                     [6],
                     [4],
                     [7],
                     [5],
                     [7],
                     [2],
                     [1],
                     [5],
                     [4],
                     [9],
                     [9],
                     [7],
                     [9],
                     [7],
                     [4],
                     [3],
                     [0],
                     [7]], dtype=int32))])

In [6]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

In [20]:
sample_clients = emnist_train.client_ids[0:100]    # <--- CHANGE This

federated_train_data = make_federated_data(emnist_train, sample_clients)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

Number of client datasets: 100
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 784)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>


In [24]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [22]:
NUM_ROUNDS=25   # <--- Change This :::  Numberof Rounds kept to 5 to save time 
logdir = "/Users/pramitdutta/opt/TensorflowApps/federatedPD/logs/trainSGD_C100_R25/"    # <--- CHANGE THIS C= Clients R= Rounds

# The following is with a single set of Sample Clients

In [23]:
#@test {"skip": true}
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()
#@test {"skip": true}
with summary_writer.as_default():
    for round_num in range(1, NUM_ROUNDS):
        state, metrics = iterative_process.next(state, federated_train_data)
        print('round {:2d}, metrics={}'.format(round_num, metrics))
        for name, value in metrics['train'].items():
            tf.summary.scalar(name, value, step=round_num)

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.11836302), ('loss', 3.091023)]))])
round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.13341816), ('loss', 3.0021853)]))])
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.15642494), ('loss', 2.862499)]))])
round  4, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.17510602), ('loss', 2.724612)]))])
round  5, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.19717981), ('loss', 2.6061547)]))])
round  6, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.22126803), ('loss', 2.5220118)]))])
round  7, metrics=O

# The following is with Random Set of Sample Clients


In [25]:
import random

#   # <--- CHANGE This
clientsPerRound = 100
clientsPassed = []  # List of clients those have been completed
#clientsCurrent = [] # List of clients in the current round

#print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
#print('First dataset: {d}'.format(d=federated_train_data[0]))

NUM_ROUNDS=25   # <--- Change This :::  Numberof Rounds kept to 25 to save time 
logdir = "/Users/pramitdutta/opt/TensorflowApps/federatedPD/logs/trainSGD_CRAND100_R25/"    # <--- CHANGE THIS CRAND= Random Clients R= Rounds 

summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()


with summary_writer.as_default():
    for round_num in range(1, NUM_ROUNDS):
        
        clientsCurrent = random.sample(emnist_train.client_ids, clientsPerRound)
        print(clientsCurrent)
        federate_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(ID))for ID in clientsCurrent]
        
        state, metrics = iterative_process.next(state, federated_train_data)
        
        print('round {:2d}, metrics={}'.format(round_num, metrics))
        for name, value in metrics['train'].items():
            tf.summary.scalar(name, value, step=round_num)
        clientsCurrent=[]


['f0577_19', 'f0533_32', 'f2326_83', 'f1608_37', 'f1064_32', 'f0284_34', 'f3699_41', 'f1121_47', 'f3209_08', 'f2165_54', 'f2175_59', 'f0509_28', 'f1160_23', 'f1668_08', 'f2289_62', 'f3718_40', 'f3476_46', 'f0913_31', 'f3597_18', 'f2438_76', 'f1269_14', 'f3529_11', 'f0871_23', 'f0933_38', 'f3164_09', 'f2546_62', 'f0452_37', 'f3115_25', 'f1358_13', 'f3454_44', 'f1476_39', 'f1631_40', 'f0924_33', 'f2582_70', 'f1018_07', 'f2163_51', 'f3492_35', 'f0804_00', 'f1784_17', 'f3543_29', 'f1489_03', 'f2365_62', 'f0952_49', 'f1433_36', 'f0740_43', 'f3909_46', 'f2432_94', 'f2216_66', 'f3383_24', 'f3944_02', 'f0291_06', 'f2025_00', 'f1787_36', 'f1190_46', 'f2391_85', 'f3758_19', 'f2285_54', 'f0918_47', 'f1838_49', 'f1707_05', 'f0985_44', 'f2363_77', 'f1767_34', 'f1643_46', 'f1869_07', 'f2448_94', 'f2103_60', 'f1429_14', 'f3496_38', 'f1514_35', 'f1486_35', 'f0981_17', 'f1371_21', 'f0678_26', 'f4089_12', 'f1751_08', 'f0764_08', 'f1204_09', 'f2533_94', 'f4029_29', 'f0719_05', 'f0953_15', 'f3201_07', 'f1

round  7, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.24270569), ('loss', 2.430144)]))])
['f4013_32', 'f3484_17', 'f2506_77', 'f1962_12', 'f3761_12', 'f2102_52', 'f1749_23', 'f0591_27', 'f2074_03', 'f0629_39', 'f3332_19', 'f4017_11', 'f3162_20', 'f1688_24', 'f0446_03', 'f0625_26', 'f3414_29', 'f2112_65', 'f1987_47', 'f1836_34', 'f3357_06', 'f2152_98', 'f0729_08', 'f1358_13', 'f0122_45', 'f1284_38', 'f2414_89', 'f0522_38', 'f2218_51', 'f0479_35', 'f0308_10', 'f4082_29', 'f3720_15', 'f1946_25', 'f1945_00', 'f0243_29', 'f2257_69', 'f2559_95', 'f1659_04', 'f0397_27', 'f0275_17', 'f0821_44', 'f2282_53', 'f1938_21', 'f3166_12', 'f0663_30', 'f1008_28', 'f3845_39', 'f1590_15', 'f2358_71', 'f3980_34', 'f1810_02', 'f1879_44', 'f2227_89', 'f0676_08', 'f0797_22', 'f3670_40', 'f3101_07', 'f1396_07', 'f2144_67', 'f3982_48', 'f2547_96', 'f0582_44', 'f2456_71', 'f0613_28', 'f0355_32', 'f1614_22', 'f1222_04', 'f1353_42', 'f3286_4

round 14, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.3991306), ('loss', 1.846223)]))])
['f3988_14', 'f3347_47', 'f4044_44', 'f4033_05', 'f1547_06', 'f0301_33', 'f3404_40', 'f3225_26', 'f3728_28', 'f0784_30', 'f1788_32', 'f1378_08', 'f0702_46', 'f1776_35', 'f1546_05', 'f0481_47', 'f1877_30', 'f3438_08', 'f3757_19', 'f0267_24', 'f3916_23', 'f3271_09', 'f4084_14', 'f1952_17', 'f2354_87', 'f3502_45', 'f1117_18', 'f3307_33', 'f3292_31', 'f1631_40', 'f0558_01', 'f1870_30', 'f1687_40', 'f2039_13', 'f2174_61', 'f0909_04', 'f1481_35', 'f0508_20', 'f0087_24', 'f0477_46', 'f3640_13', 'f3629_19', 'f0892_23', 'f0701_21', 'f1794_07', 'f1007_22', 'f0996_01', 'f1513_07', 'f2086_25', 'f3235_00', 'f2433_75', 'f1217_02', 'f0176_30', 'f1747_44', 'f0905_43', 'f3712_14', 'f2382_52', 'f1436_39', 'f2110_56', 'f2056_40', 'f2451_68', 'f3958_42', 'f0971_24', 'f1972_15', 'f1377_09', 'f0452_37', 'f1568_21', 'f1595_34', 'f0359_05', 'f3254_32

round 21, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.531531), ('loss', 1.4861575)]))])
['f3259_15', 'f3876_02', 'f3518_46', 'f3632_10', 'f3105_03', 'f2381_54', 'f1988_11', 'f1805_13', 'f2011_41', 'f1203_10', 'f3761_12', 'f2208_81', 'f0644_19', 'f2431_93', 'f0819_13', 'f2140_77', 'f3382_25', 'f1848_43', 'f3641_13', 'f3936_41', 'f1688_24', 'f1364_24', 'f0225_20', 'f2233_79', 'f0527_23', 'f0637_21', 'f1508_14', 'f0895_48', 'f2459_55', 'f2496_56', 'f1772_12', 'f3148_21', 'f0950_21', 'f3461_23', 'f1669_41', 'f3299_12', 'f0094_41', 'f3368_37', 'f0876_43', 'f0344_25', 'f3629_19', 'f0566_02', 'f1902_10', 'f0928_42', 'f0915_03', 'f3201_07', 'f0194_30', 'f2508_71', 'f1528_18', 'f1010_11', 'f0572_15', 'f1894_39', 'f1970_32', 'f3750_00', 'f0454_20', 'f0521_14', 'f3888_32', 'f3796_14', 'f1430_10', 'f1269_14', 'f0861_29', 'f1544_46', 'f3306_39', 'f1164_21', 'f1509_45', 'f1862_37', 'f3619_19', 'f1550_34', 'f2032_22', 'f1847_26

# Round Robin Method

In [26]:

#   # <--- CHANGE This
clientsPerRound = 100

NUM_ROUNDS=25   # <--- Change This :::  Numberof Rounds kept to 25 to save time 
logdir = "/Users/pramitdutta/opt/TensorflowApps/federatedPD/logs/trainSGD_CRR100_R25/"    # <--- CHANGE THIS CRAND= Random Clients R= Rounds 

summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()


with summary_writer.as_default():
    for round_num in range(1, NUM_ROUNDS):
        
        sample_clients = emnist_train.client_ids[(round_num-1)*100+1:round_num*100]    # <--- CHANGE This
        federated_train_data = make_federated_data(emnist_train, sample_clients)

        state, metrics = iterative_process.next(state, federated_train_data)
        
        print('round {:2d}, metrics={}'.format(round_num, metrics))
        for name, value in metrics['train'].items():
            tf.summary.scalar(name, value, step=round_num)
        clientsCurrent=[]



round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.11851376), ('loss', 3.0874672)]))])
round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.12858643), ('loss', 2.9518359)]))])
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.15812607), ('loss', 2.8483715)]))])
round  4, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.17114905), ('loss', 2.7295804)]))])
round  5, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.19602731), ('loss', 2.6081629)]))])
round  6, metrics=OrderedDict([('broadcast', ()), ('aggregation', ()), ('train', OrderedDict([('sparse_categorical_accuracy', 0.22336182), ('loss', 2.4771786)]))])
round  7, metric

In [2]:
%tensorboard --logdir /Users/pramitdutta/opt/TensorflowApps/federatedPD/logs/ --port=0

In [80]:
!rm -R /tmp/logs/scalars/trainSGD_CRAND25_R25/

In [35]:
k = [0,1,2,3,4]

In [36]:
print(k)

[0, 1, 2, 3, 4]


In [45]:
for i in range (0,5): 
    print(int(k[i]))
    

0
1
2
3
4


In [48]:
k[:]

[0, 1, 2, 3, 4]

In [ ]:

#def getClient():
#    clientsCurrent = []
#    for i in range(clientsPerRound):
#        newClientID = random.randint(1,3382)
#        if newClientID not in clientsPassed: 
#            clientsPassed.append(newClientID)
#            clientsCurrent.append(newClientID)
#    print(clientsCurrent)
#    return clientsCurrent
        #sample_clients = emnist_train.client_ids[getClientID()]
        
        # Since Nested List is NOT possible with the make_federate_data
        # This is changed to the new function make_client_data
        #clients = getClient()
        #print(clients)